In [1]:
import numpy as np

In [2]:
positionz = np.load("positions.npz")

In [3]:
positionz.files

['L', 'R']

In [4]:
L = positionz["L"]
R = positionz["R"]

In [8]:
L.ndim

2

In [9]:
arr = L

In [10]:
t0 = arr[:,0] # get the first column t=0

In [15]:
subbed = np.subtract(arr[:,:].T,t0).T # subtract first column from all columns

In [16]:
subbed

array([[ 0.000e+00, -1.000e+00, -1.000e+00, ..., -5.607e+03, -5.608e+03,
        -5.608e+03],
       [ 0.000e+00, -1.000e+00, -2.000e+00, ..., -3.848e+03, -3.848e+03,
        -4.264e+03],
       [ 0.000e+00,  1.321e+03, -2.000e+00, ...,  3.424e+03,  1.097e+03,
         1.097e+03],
       ...,
       [ 0.000e+00,  1.000e+00,  1.000e+00, ...,  9.389e+03,  7.510e+03,
         9.255e+03],
       [ 0.000e+00, -1.000e+00, -1.000e+00, ...,  1.609e+03,  1.609e+03,
         8.946e+03],
       [ 0.000e+00,  0.000e+00, -1.000e+00, ...,  7.446e+03,        nan,
         1.565e+03]])

In [40]:
linear_distance = np.abs(subbed) # take the absolute value to get distance

In [41]:
linear_distance[np.isnan(linear_distance)] = -1 # convert nan to -1
linear_distance = linear_distance.astype(np.int16) # convert to i16 for the id 

In [42]:
linear_distance

array([[   0,    1,    1, ..., 5607, 5608, 5608],
       [   0,    1,    2, ..., 3848, 3848, 4264],
       [   0, 1321,    2, ..., 3424, 1097, 1097],
       ...,
       [   0,    1,    1, ..., 9389, 7510, 9255],
       [   0,    1,    1, ..., 1609, 1609, 8946],
       [   0,    0,    1, ..., 7446,   -1, 1565]], dtype=int16)

In [ ]:
status = linear_distance.copy()

In [44]:
linear_distance[(linear_distance>1)] = -1 # convert unbounds to -1

In [45]:
linear_distance

array([[ 0,  1,  1, ..., -1, -1, -1],
       [ 0,  1, -1, ..., -1, -1, -1],
       [ 0, -1, -1, ..., -1, -1, -1],
       ...,
       [ 0,  1,  1, ..., -1, -1, -1],
       [ 0,  1,  1, ..., -1, -1, -1],
       [ 0,  0,  1, ..., -1, -1, -1]], dtype=int16)